In [2]:
import pandas as pd
import re
from io import StringIO

import os
import sys
import urllib
import json
import googletrans


def back_translate_google(source_text):
    translator = googletrans.Translator()
    
    # en와 ja 비슷, zh-cn와 zh-tw 비슷
    lang_list = ["en", "zh-cn"]
    back_trans_list = list()
    
    # print(googletrans.LANGUAGES)
    for lang in lang_list:
        trans_text = translator.translate(source_text, dest=lang)
        back_trans_text = translator.translate(trans_text.text, dest="ko")
        back_trans_list.append(back_trans_text.text)
    
    return back_trans_list

def back_translate_papago(source_text):
    # 보안상 개인 naver api key 사용 바람
    json_file = open("./translator_key.json", encoding="utf-8")
    key_dict = json.loads(json_file.read())

    client_id = key_dict["client_id"]
    client_secret = key_dict["client_secret"]

    url = "https://openapi.naver.com/v1/papago/n2mt"

    lang_list = ["en", "ja"]
    
    target_lang = lang_list[0]
    ### 한국어 -> 외국어
    enc_text = urllib.parse.quote(source_text)
    data = f"source=ko&target={target_lang}&text=" + enc_text
    
    # 요청 header 및 parameter
    req_header = {"X-Naver-Client-Id":client_id, "X-Naver-Client-Secret":client_secret}
    req_parameter = {"source":"ko", "target":target_lang, "text":encText}
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()

    if(rescode == 200):
        response_body = response.read()
        res_json = json.loads(response_body.decode("utf-8"))
        trans_text = res_json["message"]["result"]["translatedText"]
    else:
        print("error Code:" + rescode)
        sys.exit(1)
    
    ### 외국어 -> 한국어
    enc_text = urllib.parse.quote(trans_text)
    data = f"source={target_lang}&target=ko&text=" + enc_text
    
    # 요청 header 및 parameter
    req_parameter = {"source":target_lang, "target":"ko", "text":trans_text}
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()

    if(rescode == 200):
        response_body = response.read()
        res_json = json.loads(response_body.decode("utf-8"))
        back_trans_text = res_json["message"]["result"]["translatedText"]
    else:
        print("error Code:" + rescode)
        sys.exit(1)
    
    print(source_text)
    print("-" * 30)
    print(trans_text)
    print("-" * 30)
    print(back_trans_text)
    
    return back_trans_text

def main():    
    origin_strongbuy = pd.read_csv("./data/strongbuy.csv")
    origin_sell = pd.read_csv("./data/sell.csv")
    origin_holddown = pd.read_csv("./data/holddown.csv")
    
    print(origin_strongbuy)

    # back_trans_text = back_translate_papago(preprocessed_text_data)
    
    
if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'googletrans'